In [23]:
import pandas as pd

mslAndSST = pd.read_csv('msl_and_sst.csv')
pressure = pd.read_csv('pressure.csv')

In [24]:
# Merge the dataframes based on latitude, longitude, and valid_time
merged_df = pd.merge(mslAndSST, pressure, on=['latitude', 'longitude', 'valid_time'], how='inner')
print(merged_df.head())

In [27]:
'''
sst > 299.65
Latitude between 5° and 30°
msl < 101_000
wind shear < 10
relative humidity > 60
Vorticity > 10^-5
'''
df = merged_df[merged_df['sst'] > 299.65]
df = df[df['msl'] < 101_000]
df = df[df['vertical_wind_shear'] < 10]
df = df[df['relative_humidity'] > 60]
df = df[df['vorticity'] > 10**-5]
df = df[(df['latitude'] > 5) & (df['latitude'] < 45)]
df = df[(df['longitude'] < -10) & (df['longitude'] > -100)]
df

,latitude,longitude,valid_time,msl,sst,vertical_wind_shear,relative_humidity,vorticity
639456,6.0,-99.0,2020-05-07 00:00:00,100857.0,302.0,6.353390,70.682290,0.000014
639508,6.0,-99.0,2020-05-20 00:00:00,100948.0,302.0,4.723331,91.982090,0.000015
639510,6.0,-99.0,2020-05-20 12:00:00,100979.0,302.0,3.085481,91.525840,0.000110
639512,6.0,-99.0,2020-05-21 00:00:00,100929.0,302.0,6.755966,78.600490,0.000042
639750,6.0,-99.0,2020-07-19 12:00:00,100904.0,301.0,8.187444,65.447105,0.000018
...,...,...,...,...,...,...,...,...
4295842,41.0,-63.0,2020-08-17 12:00:00,100957.0,301.0,5.334395,92.450970,0.000020
4295843,41.0,-63.0,2020-08-17 18:00:00,100620.0,301.0,7.991423,63.981960,0.000032
4300177,41.0,-58.0,2020-08-31 06:00:00,100860.0,300.0,7.764542,82.190010,0.000041
4303542,41.0,-54.0,2020-08-16 12:00:00,100931.0,300.0,9.093582,75.314610,0.000055


In [29]:
df.to_csv('filtered_data.csv', index=False)